In [ ]:
import json
import requests
from requests import Response
from pprint import pprint
from pydantic import (
    BaseModel,
    RootModel,
    Field,
    ValidationError,
    field_validator
    )
import pyinputplus as pipy
from random import choices

class UBike(BaseModel):
    YouBike站所名稱:str = Field(alias = 'sna')
    站所區域:str = Field(alias = 'sarea')
    資料更新時間:str = Field(alias = 'mday')
    地址:str = Field(alias = 'ar')
    act:int
    站所總停車格:int = Field(alias = 'total')
    站所目前車輛數:int = Field(alias = 'available_rent_bikes')
    空位數量:int = Field(alias = 'available_return_bikes')
    緯度:float = Field(alias = 'latitude')
    經度:float = Field(alias = 'longitude')
    
    
class Root(RootModel):
    root:list[UBike]
    
def main():
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    res:Response = requests.get(youbike_url)
    all_data = res.json()
    
    try:
        root:Root = Root.model_validate(all_data)
    except ValidationError as error:
        print(error)
        
    data_list:list[dict] = root.model_dump()
    filter_list:list[dict] = list(filter(lambda item:False if item['act'] == 0 else True,data_list))
    filter_list_rent_less3:list[dict] = list(filter(lambda item:True if item['站所目前車輛數'] < 3 else False,data_list))
    filter_list_return_less3:list[dict] = list(filter(lambda item:True if item['空位數量']<3 else False,data_list))
    
    print(f"台北市Youbike有{len(data_list)}營運站,目前有{len(filter_list)}站正常營運")
    
    num = pipy.inputInt("請輸入隨機顯示的站點數(1~5): ", min=1, max=5)
    print(num)
    
    print(f"\n可借車輛數為3輛以內的站點有:{len(filter_list_rent_less3)}站")
    num_list:list[dict] = choices(filter_list_rent_less3,k = num)
    
    
    
    
    
    
if __name__ == __main__:
    main()
    